In [1]:
#!/usr/bin/env python3
"""
Consolida CSVs de simulação de chuveiros em um único Parquet.
- Lê arquivos no padrão: shower_hHH_tTT_1min.csv (ex.: shower_h00_t15_1min.csv)
- Faz parsing de HH (hora) e TT (threshold °C) a partir do nome
- Gera um único arquivo .parquet com todos os registros
"""

import re
import os
import glob
import pandas as pd

# === CONFIGURAÇÕES ===
INPUT_DIR = "Output"                 # pasta onde estão os CSVs
GLOB_PATTERN = "shower_h*_t*_1min.csv"
OUTPUT_PARQUET = "shower_all.parquet"
# Se quiser salvar no próprio "Output", use: OUTPUT_PARQUET = os.path.join(INPUT_DIR, "shower_all.parquet")

# Regex para extrair hora e threshold do nome do arquivo
FILENAME_RE = re.compile(r"shower_h(\d{2})_t(\d+)_1min\.csv$", re.IGNORECASE)

# Colunas que certamente são numéricas (ajuste se necessário)
NUMERIC_COLS = [
    "latitude", "longitude", "year", "bath_hours", "bath_minutes", "flow_lpm", "efficiency",
    "temp_limit_c", "temp_usage_target_c", "consumo_anual_kwh", "consumo_dia_max_kwh",
    "required_kw_p95", "nominal_kw_p95", "required_kw_max", "nominal_kw_max"
]

def parse_from_filename(path: str):
    """Retorna (hour, threshold) extraídos do nome do arquivo."""
    name = os.path.basename(path)
    m = FILENAME_RE.search(name)
    if not m:
        raise ValueError(f"Arquivo fora do padrão esperado: {name}")
    hour = int(m.group(1))
    threshold = int(m.group(2))
    return hour, threshold

def read_csv_safe(path: str) -> pd.DataFrame:
    """Lê CSV com fallback de encoding e tipos."""
    try:
        df = pd.read_csv(path, low_memory=False)
    except UnicodeDecodeError:
        df = pd.read_csv(path, low_memory=False, encoding="latin-1")

    # Normaliza nomes de colunas (tira espaços/brancos, mantém case original)
    df.columns = [c.strip() for c in df.columns]

    # Converte colunas numéricas conhecidas quando existirem
    for col in NUMERIC_COLS:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # Alguns campos de texto podem vir como numéricos por engano; força para string onde faz sentido
    for text_col in ["city", "state", "scenario", "temp_usage_mode", "power_selection"]:
        if text_col in df.columns:
            df[text_col] = df[text_col].astype(str).str.strip()

    return df

def main():
    input_path = os.path.abspath(INPUT_DIR)
    files = sorted(glob.glob(os.path.join(input_path, GLOB_PATTERN)))
    if not files:
        raise SystemExit(f"Nenhum CSV encontrado em {input_path} com padrão {GLOB_PATTERN}")

    frames = []
    for i, f in enumerate(files, 1):
        hour, threshold = parse_from_filename(f)
        df = read_csv_safe(f)
        # adiciona metadados vindos do nome do arquivo
        df["sim_hour"] = hour
        df["temp_threshold_c"] = threshold
        df["sim_minutes_per_day"] = 1  # pelo padrão 1min no nome
        df["source_file"] = os.path.basename(f)
        frames.append(df)

        if i % 50 == 0 or i == len(files):
            print(f"Lidos {i}/{len(files)} arquivos...")

    full = pd.concat(frames, ignore_index=True)

    # Ordena colunas: meta primeiro, depois o restante
    meta_cols = ["city", "state", "latitude", "longitude", "year", "scenario",
                 "sim_hour", "temp_threshold_c", "sim_minutes_per_day", "source_file"]
    ordered_cols = meta_cols + [c for c in full.columns if c not in meta_cols]
    full = full[ordered_cols]

    # Tipos finais (onde fizer sentido)
    for col in ["sim_hour", "temp_threshold_c", "sim_minutes_per_day", "year"]:
        if col in full.columns:
            full[col] = pd.to_numeric(full[col], errors="coerce").astype("Int64")

    # Salva Parquet (único arquivo)
    full.to_parquet(OUTPUT_PARQUET, engine="pyarrow", index=False)
    print(f"✅ Consolidado com {len(full):,} linhas em: {os.path.abspath(OUTPUT_PARQUET)}")

    # Pequeno resumo
    try:
        resumo = (
            full.groupby(["sim_hour", "temp_threshold_c"])
                .size()
                .rename("linhas")
                .reset_index()
                .sort_values(["sim_hour", "temp_threshold_c"])
        )
        print("\nAmostra do resumo por hora/threshold:")
        print(resumo.head(10).to_string(index=False))
    except Exception:
        pass

if __name__ == "__main__":
    main()


Lidos 50/624 arquivos...
Lidos 100/624 arquivos...
Lidos 150/624 arquivos...
Lidos 200/624 arquivos...
Lidos 250/624 arquivos...
Lidos 300/624 arquivos...
Lidos 350/624 arquivos...
Lidos 400/624 arquivos...
Lidos 450/624 arquivos...
Lidos 500/624 arquivos...
Lidos 550/624 arquivos...
Lidos 600/624 arquivos...
Lidos 624/624 arquivos...
✅ Consolidado com 3,246,048 linhas em: c:\Users\igorc\Desktop\DocumentosAT\Repo Local\water-heating-analysis\shower_all.parquet

Amostra do resumo por hora/threshold:
 sim_hour  temp_threshold_c  linhas
        0                15    5202
        0                16    5202
        0                17    5202
        0                18    5202
        0                19    5202
        0                20    5202
        0                21    5202
        0                22    5202
        0                23    5202
        0                24    5202
